In [3]:
import os
import pandas as pd 
import traceback
import json

In [4]:
import os
os.environ["OPENAI_API_KEY"] = "OPENROUTERAI_API_KEY"

In [11]:
from langchain_openai import ChatOpenAI

In [12]:
class ChatOpenRouter(ChatOpenAI):
    """Subclass of ChatOpenAI that uses OpenRouter API key from environment"""


In [13]:
from dotenv import load_dotenv

In [14]:
load_dotenv()

True

In [16]:
llm = ChatOpenAI(
    model_name="qwen/qwen3-coder-next",
    openai_api_base="https://openrouter.ai/api/v1",
    openai_api_key=os.environ["OPENROUTERAI_API_KEY"],
    temperature=0.7,
    max_tokens=2000
)

In [18]:
llm.invoke("who is the ceo of google?")

AIMessage(content="The CEO of Google is **Sundar Pichai**. He has held the position since October 2015, when he became CEO of Google Inc. Following the 2015 restructuring of Google into Alphabet Inc., he continued as CEO of Google. In December 2023, he also became CEO of **Alphabet Inc.**, the parent company of Google and its subsidiaries.\n\nLet me know if you'd like more details about his background or role!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 100, 'prompt_tokens': 16, 'total_tokens': 116, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}, 'cost': 0.000128, 'is_byok': False, 'cost_details': {'upstream_inference_cost': 0.000128, 'upstream_inference_prompt_cost': 8e-06, 'upstream_inference_completions_cost': 0.00012}}, 'model_provider': 'openai', 'model_name': 'qwen/qwen

In [34]:
# ================================
# Imports & Environment Setup
# ================================
import os
from dotenv import load_dotenv
import PyPDF2
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI


In [35]:
# Load API key from .env or set directly
load_dotenv()
OPENAI_API_KEY = os.environ.get("OPENROUTERAI_API_KEY")  # your real API key here

In [ ]:
# ================================
# PDF Text Extraction
# ================================
def extract_pdf_text(pdf_path: str) -> str:
    """
    Extract all text from a PDF file using PyPDF2
    """
    text = ""
    with open(pdf_path, "rb") as f:
        reader = PyPDF2.PdfReader(f)
        for page in reader.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text + "\n"
    return text


In [38]:
# ================================
# Chat Prompt Template
# ================================
mcq_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an expert exam question generator."),
    ("user", """
Generate {num_mcqs} multiple-choice questions at {difficulty} difficulty 
from the following text:

{pdf_text}

Output format:
1. Question?
   a) Option A
   b) Option B
   c) Option C
   d) Option D
   Answer: <correct option letter>

Generate only {num_mcqs} questions.
""")
])



In [48]:
# ================================
# Initialize OpenRouter LLM
# ================================
llm = ChatOpenAI(
    model_name="qwen/qwen3-coder-next",
    openai_api_base="https://openrouter.ai/api/v1",
    openai_api_key=OPENAI_API_KEY,
    temperature=1.0,
    max_tokens=2000
)



In [44]:
def generate_mcqs_from_pdf(pdf_path: str, num_mcqs: int = 5, difficulty: str = "medium") -> str:
    """
    Extract text from PDF, format prompt, and generate MCQs using OpenRouter LLM.
    """
    # 1️⃣ Extract PDF text
    pdf_text = extract_pdf_text(pdf_path)
    if not pdf_text.strip():
        return "No text found in PDF."

    # 2️⃣ Format prompt
    messages = mcq_prompt.format(
        pdf_text=pdf_text,
        num_mcqs=num_mcqs,
        difficulty=difficulty
    )

    # 3️⃣ Generate MCQs using invoke()
    response = llm.invoke(messages)

    return response.content


In [49]:
# ================================
# Example Usage
# ================================
if __name__ == "__main__":
    pdf_file = r"D:\Gen Ai\mcqgen\examplePdf\AttentionResearch.pdf"  # raw string for Windows path
    mcqs = generate_mcqs_from_pdf(pdf_file, num_mcqs=5, difficulty="very easy")
    print(mcqs)

1. What must the traveler present at the check-in counter?  
   a) Arabic version of the e-ticket  
   b) Printed boarding pass only  
   c) English version of the e-ticket  
   d) Passport copy only  
   Answer: c  

2. How much checked baggage allowance is included for Mr. Ibne Ahmad on the DMM to LHE flight?  
   a) 30 kg (1 piece only)  
   b) 60 kg (2 pieces)  
   c) 7 kg (1 piece)  
   d) 10 kg (1 piece)  
   Answer: b  

3. What is the cancellation fee for the Lahore to Dammam (Fly Jinnah) flight?  
   a) SAR 150 + fare difference  
   b) SAR 200 + applicable taxes  
   c) SAR 35 (Almosafer fee only)  
   d) SAR 0 + applicable taxes  
   Answer: d  

4. When should international passengers arrive at the airport?  
   a) At least 1 hour before departure  
   b) At least 2 hours before departure  
   c) At least 3 hours before departure  
   d) At least 4 hours before departure  
   Answer: d  

5. Which airline operates the return flight from Lahore to Dammam?  
   a) Pakistan In